In [27]:
import numpy as np
import pandas as pd


# Retrieve COVID data from ECDC 

In [2]:
covid_url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import json
import urllib

In [3]:
covid_json_unformated = urllib.request.urlopen(covid_url).read().decode("utf-8")
covid_json = json.loads(covid_json_unformated)
cdf = pd.DataFrame(covid_json['records'])

In [4]:
cdf.sample(10)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
33080,18/08/2020,18,08,2020,0,0,Lesotho,LS,LSO,2125267.0,Africa,10.72806382
19585,15/08/2020,15,08,2020,0,0,Faroe_Islands,FO,FRO,48677.0,Europe,281.44709
35377,22/06/2020,22,06,2020,16,0,Malaysia,MY,MYS,31949789.0,Asia,0.78247778
46907,11/01/2020,11,01,2020,0,0,Romania,RO,ROU,19414458.0,Europe,
20281,26/03/2020,26,03,2020,88,2,Finland,FI,FIN,5517919.0,Europe,14.8787976
13988,26/05/2020,26,05,2020,0,1,Croatia,HR,HRV,4076246.0,Europe,1.17755405
54224,24/03/2020,24,03,2020,182,11,Sweden,SE,SWE,10230185.0,Europe,17.22353995
49733,07/05/2020,07,05,2020,114,3,Serbia,RS,SRB,6963764.0,Europe,38.44185415
15526,04/05/2020,04,05,2020,10,1,Democratic_Republic_of_the_Congo,CD,COD,86790568.0,Africa,0.40557403
10894,30/04/2020,30,04,2020,0,0,Cape_Verde,CV,CPV,549936.0,Africa,10.5466818


In [5]:
cdf.shape

(61900, 12)

In [17]:
cdf.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,date_reported
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.01377925,2020-12-14
1,13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.05277624,2020-12-13
2,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.86876792,2020-12-12
3,11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.13426564,2020-11-12
4,10/12/2020,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.96865815,2020-10-12


Rename columns to something more Pythonian. If you think they look already great, then at least rename `notification_rate_per_100000_population_14-days` to `14d-incidence`

In [18]:
cdf.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000',
       'date_reported'],
      dtype='object')

In [19]:
cdf.rename(columns={'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000':'14d-incidence'})

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,14d-incidence,date_reported
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.01377925,2020-12-14
1,13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.05277624,2020-12-13
2,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.86876792,2020-12-12
3,11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.13426564,2020-11-12
4,10/12/2020,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.96865815,2020-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61895,25/03/2020,25,03,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,,2020-03-25
61896,24/03/2020,24,03,2020,0,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,,2020-03-24
61897,23/03/2020,23,03,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,,2020-03-23
61898,22/03/2020,22,03,2020,1,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,,2020-03-22


Identify which columns have not been casted to an appropriate type during loading!

In [20]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61900 entries, 0 to 61899
Data columns (total 13 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   dateRep                                                     61900 non-null  object        
 1   day                                                         61900 non-null  object        
 2   month                                                       61900 non-null  object        
 3   year                                                        61900 non-null  object        
 4   cases                                                       61900 non-null  int64         
 5   deaths                                                      61900 non-null  int64         
 6   countriesAndTerritories                                     61900 non-null  object        
 7   geoId                 

We did not cover datetime objects in pandas, however they are quite powerful!

Try:

In [21]:
cdf['date_reported'] = pd.to_datetime(cdf['dateRep'])

Now you can treat the column as a datetime objects using `df[col].dt` , e.g. https://docs.python.org/3/library/datetime.html#datetime.date.year

In [22]:
cdf['date_reported'].dt.day.head()

0    14
1    13
2    12
3    12
4    12
Name: date_reported, dtype: int64

Create a new column `deltaTime_since_start_of_recording`

In [28]:
delta_time = []
cdf["deltaTime_since_start_of_recording"] = np.nan

Create histograms for different columns or describe the df. Can you spot the inconsistency in the data? Fix it! :)

Identify those countries (grouped by continent) which showed the most drastic increase most drastic and decrease of the `14d-incidence` within the different years since recording. Visualize intuitively!

Which country showed the highest/lowest fluctuation in `14d-incidence` within a year?

Create a line plot showing the `14-incidence` for all European countries. Use `groupby` operation to generate the data list for the plotly plot. 

Create a smoothed version of the `14d-incidence` by averaging 3 months.

Create a radial plot of death rate / 100000 people (see popData2019), where one year completes a circle, i.e. 360˚. Visualize the recored years for Italy, Germany, Sweden and Greece. Hint you might need to turn the dateTime into `day within the year` (%j) and adjust 365 to 360 degrees. 

Optional: Find "regular" mortality rates for those countries and visualize it in the plot as well.